In [72]:
import pandas as pd
from collections import Counter
from sklearn.utils import shuffle
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
import email
import re
from collections import Counter
from sklearn.utils import shuffle
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim import corpora, models
import gensim
import numpy as np


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score



# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *
from sklearn.feature_extraction.text import CountVectorizer

# Other libraries
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

In [5]:
emails = pd.read_csv("emails.csv")

In [6]:
print(emails.shape)

(517401, 2)


In [7]:
print("This dataset has",emails.shape[0],"records and",emails.shape[1],"features")

This dataset has 517401 records and 2 features


In [8]:
emails.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [9]:
print(emails["message"][1])

Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the

In [10]:
fils = emails['file']
sub_fils = [x.split('/')[0] for x in fils]
print(set(sub_fils))

{'quigley-d', 'buy-r', 'lucci-p', 'delainey-d', 'mclaughlin-e', 'perlingiere-d', 'rapp-b', 'south-s', 'storey-g', 'geaccone-t', 'campbell-l', 'steffes-j', 'carson-m', 'staab-t', 'reitmeyer-j', 'whalley-l', 'beck-s', 'gilbertsmith-d', 'grigsby-m', 'lenhart-m', 'harris-s', 'jones-t', 'nemec-g', 'sanchez-m', 'sager-e', 'donohoe-t', 'lay-k', 'horton-s', 'badeer-r', 'griffith-j', 'linder-e', 'saibi-e', 'tycholiz-b', 'fossum-d', 'stclair-c', 'merriss-s', 'zipper-a', 'meyers-a', 'schoolcraft-d', 'stokley-c', 'mims-thurston-p', 'rodrique-r', 'kuykendall-t', 'dasovich-j', 'skilling-j', 'ward-k', 'wolfe-j', 'swerzbin-m', 'arnold-j', 'semperger-c', 'hendrickson-s', 'tholt-j', 'allen-p', 'heard-m', 'kaminski-v', 'arora-h', 'white-s', 'presto-k', 'mccarty-d', 'panus-s', 'donoho-l', 'lokey-t', 'phanis-s', 'gay-r', 'giron-d', 'rogers-b', 'hernandez-j', 'thomas-p', 'parks-j', 'whitt-m', 'martin-t', 'schwieger-j', 'smith-m', 'davis-d', 'keiser-k', 'crandell-s', 'lavorato-j', 'lewis-a', 'kean-s', 'fisch

In [66]:
allemails = pd.read_csv("emails.csv")
emails = allemails[:100]

In [67]:
def extract_data():
    df = pd.DataFrame(columns=['date',
                               'sender_address',
                               'recipient',
                               'subject',
                               'sender_name',
                               'recipient_name',
                               'cc',
                               'bcc',
                               'folder',
                               'body'])
    
    for index, details in emails.iterrows():
        raw_email_info = details['message']
        # raw_email_info = f_in.read().decode('utf8')
        date = re.findall(r'Date: (.*)', raw_email_info)[0]
        sender_address = re.findall(r'From: (.*)', raw_email_info)[0]
        recipient = re.findall(r'To: (.*)', raw_email_info)[0]
        subject = re.findall(r'Subject: (.*)', raw_email_info)[0]
        sender_name = re.findall(r'X-From: (.*)', raw_email_info)[0]
        recipient_name = re.findall(r'X-To: ([ A-Za-z]*)', raw_email_info)[0]
        cc = re.findall(r'X-cc: (.*)', raw_email_info)[0]
        bcc = re.findall(r'X-bcc: (.*)', raw_email_info)[0]
        folder = re.findall(r'"[a-zA-z-]*/(.*)/.*,"Message-ID.*>', raw_email_info)
        
        # strip everything before X-FileName
        bodies = re.findall(r'(?<=X-FileName: )(?s)(.*$)', raw_email_info)
        # Get everything after the first newline
        bodies = bodies[0].split('\n')
        body = " ".join(bodies[1:])
        
        df.loc[index] = [date, sender_address, recipient, subject, sender_name, recipient_name, cc, bcc, folder, body]
        
    print(df.shape)
    return df
                         


In [68]:
df = extract_data()

(100, 10)


In [106]:
new_df = df[
            (df.folder != '_sent_mail') &
            (df.folder != 'all_documents') &
            (df.folder != 'deleted_items') &
            (df.folder != 'inbox') &
            (df.folder != 'discussion_threads') &
            (df.folder != 'notes_inbox') &
            (df.folder != 'sent_items') &
            (df.folder != 'sent') &
            (len(df.folder) != 0)
        ]

In [128]:
data = shuffle(new_df)

In [129]:
data

,date,sender_address,recipient,subject,sender_name,recipient_name,cc,bcc,folder,body
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,Phillip K Allen,Randall L Gay,,,[],"Randy, Can you send me a schedule of the sa..."
28,"Thu, 3 May 2001 15:57:00 -0700 (PDT)",phillip.allen@enron.com,rlehmann@yahoo.com,,Phillip K Allen,rlehmann,,,[],"Reagan, Just wanted to give you an update. ..."
35,"Mon, 25 Sep 2000 07:01:00 -0700 (PDT)",phillip.allen@enron.com,ina.rangel@enron.com,,Phillip K Allen,Ina Rangel,,,[],---------------------- Forwarded by Phillip K...
95,"Mon, 21 Aug 2000 09:40:00 -0700 (PDT)",phillip.allen@enron.com,mac.d.hargrove@rssmb.com,,Phillip K Allen,mac,,,[],"Mac, Thanks for the research report on EOG...."
37,"Fri, 22 Sep 2000 00:06:00 -0700 (PDT)",phillip.allen@enron.com,kathy.moore@enron.com,,Phillip K Allen,Kathy M Moore,,,[],"Kathy, Regarding the guest password for gas ..."
56,"Fri, 8 Sep 2000 05:30:00 -0700 (PDT)",phillip.allen@enron.com,pallen70@hotmail.com,Sagewood Town Homes,Phillip K Allen,pallen,,,[],---------------------- Forwarded by Phillip K...
78,"Tue, 29 Aug 2000 05:38:00 -0700 (PDT)",phillip.allen@enron.com,mark@intelligencepress.com,,Phillip K Allen,Mark,,,[],"Mark, Were you able to log in to enron onli..."
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,Phillip K Allen,Greg Piper,,,[],Let's shoot for Tuesday at 11:45.
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,Phillip K Allen,John J Lavorato,,,[],Traveling to have a business meeting takes th...
65,"Tue, 5 Sep 2000 23:22:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,Phillip K Allen,John J Lavorato,,,[],resumes of whom?


### Cleaning the data

In [73]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
texts = []

In [112]:
for i in data['body']:
    #print "Processing",i
    # clean and tokenize document string
    tokens = tokenizer.tokenize(i)
    # remove all numbers
    tokens = [x for x in tokens if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
    # remove structural words
    tokens = [x for x in tokens if len(x) > 1]
    tokens = [x.lower() for x in tokens]
    tokens = [x for x in tokens if 'http' not in x]
    tokens = [x for x in tokens if x not in "_"]
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    # add tokens to list
    texts.append(stemmed_tokens)

dictionaryall = corpora.Dictionary(texts)

corpusall = [dictionaryall.doc2bow(text) for text in texts]

len(texts[0])

765

In [134]:
non_uni_features = []
for i in texts:
    non_uni_features += i
# print(features)
features = set(non_uni_features)
len(features)

1950

In [12]:
Top1 = Counter(sub_fils).most_common()[0]
Top2 = Counter(sub_fils).most_common()[1]
print(Top1)
print(Top2)

('kaminski-v', 28465)
('dasovich-j', 28234)


In [13]:
output = []
for ind, row in emails.iterrows():
#     if ind > 1000:
#         break
    label = row['file'].split('/')[0]
    text = row['message']
    if label == 'kaminski-v' or label == 'dasovich-j':
        output.append([text, label])

df = pd.DataFrame(output, columns = ['text', 'label'])
df = shuffle(df)
print(df.head())

                                                    text       label
43053  Message-ID: <30057866.1075856354187.JavaMail.e...  kaminski-v
22974  Message-ID: <2751714.1075851612455.JavaMail.ev...  dasovich-j
35993  Message-ID: <31913536.1075856288252.JavaMail.e...  kaminski-v
47059  Message-ID: <29088086.1075856811113.JavaMail.e...  kaminski-v
54613  Message-ID: <20032068.1075856932223.JavaMail.e...  kaminski-v


In [14]:
df.to_csv("data.csv", index = None)

In [17]:
messages = list(map(email.message_from_string, df['text']))

def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

df['content'] = list(map(get_text_from_email, messages))

In [18]:
df.head()

,text,label,content
43053,Message-ID: <30057866.1075856354187.JavaMail.e...,kaminski-v,"Vince,\n\nGreetings.\n\nI write at this time t..."
22974,Message-ID: <2751714.1075851612455.JavaMail.ev...,dasovich-j,Talked to Mark.\n\nFirst thing he has to do is...
35993,Message-ID: <31913536.1075856288252.JavaMail.e...,kaminski-v,---------------------- Forwarded by Vince J Ka...
47059,Message-ID: <29088086.1075856811113.JavaMail.e...,kaminski-v,New market we are trying to develop in Benzene...
54613,Message-ID: <20032068.1075856932223.JavaMail.e...,kaminski-v,---------------------- Forwarded by Vince J Ka...


In [130]:
num_total = len(data['body'])
train_data, train_labels = data.body[int(round(0.4 * num_total)):], data.folder[int(round(0.4 * num_total)):]
test_data, test_labels = data.body[:int(round(0.4 * num_total))], data.folder[:int(round(0.4 * num_total))]


In [131]:
print("training dataset count", train_data.shape[0])
print ("test dataset count", test_data.shape[0])

training dataset count 60
test dataset count 40


In [135]:
cv =CountVectorizer(stop_words='english',min_df=10, vocabulary=features)
train_matrix = cv.fit_transform(train_data)
test_matrix = cv.transform(test_data)

In [136]:
train_matrix.shape

(60, 1950)

In [137]:
print("Size of the vocabulary is", train_matrix.shape[1])    

print("The number of example is",train_matrix.shape[0])

Size of the vocabulary is 1950
The number of example is 60


In [138]:
features_raw = np.array(cv.get_feature_names())

features = list(np.array(list(map(lambda x: re.sub(r"[^a-zA-Z]", "", x), features_raw))))[1:]
features = list(set(features))

cv2=CountVectorizer(stop_words='english', lowercase=True,vocabulary=features, max_df=0.4)
train_matrix_post = cv2.transform(train_data)
train_matrix_post.shape

(60, 1937)

In [139]:
test_matrix = cv2.transform(test_data)

In [140]:
 ### Multinomial Naive Bayes
mnb = MultinomialNB(alpha=0.1)
mnb.fit(train_matrix_post,train_labels)
test_predict_mnb=mnb.predict(test_matrix)
print("Multinomial Naive Bayes f1 score is",f1_score(test_labels,test_predict_mnb,average='weighted'))
# print("Multinomial Naive Bayes optimal value for alpha is", mnb.best_params_)

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead.

In [144]:
### logistic regression
lr = LogisticRegression(C=0.1, multi_class= 'ovr')
lr.fit(train_matrix_post,train_labels)
test_predict_lr=lr.predict(test_matrix)
print("Logistic regression f1 score is",f1_score(test_labels,test_predict_lr,average='weighted'))

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead.